# MLP Coding Example

Acesse o servidor remoto por ssh. Crie uma virtualenv com:
```
mkvirtualenv <nome-da-sua-env>
```
Ative a sua virtualenv com:
```
workon <nome-da-sua-env>
```
Instale o jupyter:
```
pip install jupyter
```
Na pasta contendo o setup.py, instale o pacote do projeto :
```
pip install -e .
```
Comando para servir o jupyter:
```
nohup jupyter notebook --no-browser &
```
Talvez você precise de um token. Se precisar consulte com:
```
jupyter notebook list
```




Na sua máquina local, redirecione a porta adequada:
```
ssh -NfL localhost:<porta-local>:localhost:<porta-remoto> <seu-usuario>@<ip-do-servidor>
```
Geralmente:
```
ssh -NfL localhost:8888:localhost:8888 <seu-usuario>@<ip-do-servidor>
```
Abra localhost:8888 no seu browser. Se você quiser fechar o jupyter, no localhost:8888 clique em Quit, depois libere a porta com:
```
lsof -ti:8888 | xargs kill -9
```

## Imports

In [1]:
import numpy as np
from numpy import genfromtxt
from tqdm.notebook import tqdm
from perceptronac.context_training import context_training
from perceptronac.context_coding import context_coding
from perceptronac.perfect_AC import perfect_AC
import torch
import os

## Gerando dados randômicos correlacionados (substituir pelos seus dados)

In [37]:
#X is context and y is bitstream to encode
# parameters  
L = 100000 # how many samples 
N = 7 # order of the AR
# Np = N # number of parameters to estimate 

C0 = np.random.rand(1,1) 
C = np.random.rand(N,1)

X = 2 * (np.random.rand(2*L,N) > 0.5) - 1 # correlated (context) signals

X = (X > 0).astype(int)

def sigmoid(x): 
    return 1 / (1 + np.e**(-x))

p = sigmoid(C0 + X @ C);
yy = (np.random.rand(2*L, 1) > (1 - p)).astype(int) # signal 
yt = yy[0:L] > 0 # train on the first part 
yc = yy[L:L+L] > 0 # encode the second part
Xt = X[0:L,0:N] # truncated X for training 
Xc = X[L:L+L,0:N] # truncated X for coding

In [52]:
#Xc

array([[ 0, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1],
       [ 1, -1, -1, -1, -1],
       ...,
       [-1, -1,  1, -1, -1],
       [ 1, -1, -1, -1, -1],
       [-1, -1,  0, -1, -1]])

In [2]:
#X is context and y is bitstream to encode
L = 100000 # how many samples 
N = 5 # order of the AR
my_data = genfromtxt('../SPIHT_dataset/SPIHT_bits_with_context_ricardo14.csv', delimiter=',')
bitstream = my_data[:, 0].astype(int)
context = my_data[:, 1:6].astype(int)
bitstream = bitstream.reshape((len(bitstream), 1))


my_data_test = genfromtxt('../SPIHT_dataset/SPIHT_bits_with_context_ricardo15.csv', delimiter=',')
bitstream_test = my_data_test[:, 0].astype(int)
context_test = my_data_test[:, 1:6].astype(int)
bitstream_test = bitstream_test.reshape((len(bitstream_test), 1))


yt = bitstream > 0 # train on the first part 
yc = bitstream_test > 0 # encode the second part
Xt = context # truncated X for training 
Xc = context_test # truncated X for coding

In [3]:
#context

## Entropia dos dados

In [4]:
# treino
perfect_AC(yt,context_coding(Xt,context_training(Xt,yt)))

3.2034265038149176e-16

In [5]:
# teste
perfect_AC(yc,context_coding(Xc,context_training(Xc,yc)))

3.203426503814918e-16

# Criando classes

In [10]:
import torch

In [11]:
class Perceptron(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.linear = torch.nn.Linear(N, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [12]:
class Log2BCELoss(torch.nn.Module):
    def __init__(self,*args,**kwargs):
        super().__init__()
        self.bce_loss = torch.nn.BCELoss(*args,**kwargs)

    def forward(self, pred, target):
        return self.bce_loss(pred, target)/torch.log(torch.tensor(2,dtype=target.dtype,device=target.device))

In [13]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        return self.X[idx,:],self.y[idx,:]

## Treinando Modelo No Pytorch com Batch Gradient Descent (Quando todos os dados couberem na memória da placa de vídeo de uma só vez)

In [61]:
net = Perceptron(N)

In [62]:
net = net.cuda()

In [63]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [64]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)

In [65]:
train_loss = []
for epoch in range(20000):
    optimizer.zero_grad()
    outputs = net(torch.tensor(trainset.X).float().cuda())
    loss = criterion(outputs,torch.tensor(trainset.y).view(-1,1).float().cuda())
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item()/len(trainset))

In [66]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

comprimento médio de código final no dataset de treino: 0.003491003723144531
(compare com a entropia do dataset de treino).


### Pesos aprendidos são aproximadamente os parâmetros usados para gerar os dados

In [67]:
for param in net.parameters():
    print(param.data)

tensor([[12.0901, 11.7186, 11.9979, 12.1763, 12.1026]], device='cuda:0')
tensor([41.8704], device='cuda:0')


In [68]:
C.T, C0

(array([[0.32098451, 0.77295341, 0.82311742, 0.86189127, 0.7005142 ,
         0.80153656, 0.21184858]]),
 array([[0.2792105]]))

## Treinando Modelo No Pytorch com Stochastic Gradient Descent (um pedaço dos dados na memória da placa de vídeo de cada vez)

In [14]:
net = Perceptron(N)

In [15]:
net = net.cuda()

In [16]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [17]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)

In [18]:

batch_size = 100000

train_loss, valid_loss = [], []

for epoch in range(200):  # loop over the dataset multiple times

    for phase in ['train','valid']:

        if phase == 'train':
            net.train(True)
            dataloader = torch.utils.data.DataLoader(
                trainset,batch_size=batch_size,shuffle=True,num_workers=2)
        else:
            net.train(False)
            dataloader=torch.utils.data.DataLoader(
                validset,batch_size=batch_size,shuffle=False,num_workers=2)
            
        running_loss = 0.0
        for data in dataloader: #tqdm(dataloader):
            
            X_b,y_b= data
            X_b = X_b.float().cuda()
            y_b = y_b.float().cuda()
            
            if phase == 'train':
                optimizer.zero_grad()
                outputs = net(X_b.float())
                loss = criterion(outputs,y_b.view(-1,1).float())
                loss.backward()
                optimizer.step()
            else:
                with torch.no_grad():
                    outputs = net(X_b)
                    loss = criterion(outputs, y_b)

            running_loss += loss.item()

        final_loss = running_loss / len(dataloader.dataset)
        if phase=='train':
            train_loss.append(final_loss)
        else:
            valid_loss.append(final_loss)
            
        print("epoch :" , epoch, ", phase :", phase, ", loss :", final_loss)

print('Finished Training')

epoch : 0 , phase : train , loss : 0.9249599924875869
epoch : 0 , phase : valid , loss : 0.8783912045136745
epoch : 1 , phase : train , loss : 0.857345049755064
epoch : 1 , phase : valid , loss : 0.8312144911447269
epoch : 2 , phase : train , loss : 0.8135192317285984
epoch : 2 , phase : valid , loss : 0.7903413277088718
epoch : 3 , phase : train , loss : 0.7741555494609583
epoch : 3 , phase : valid , loss : 0.7527223796351283
epoch : 4 , phase : train , loss : 0.7376844539512554
epoch : 4 , phase : valid , loss : 0.7177247682844596
epoch : 5 , phase : train , loss : 0.7036943713799233
epoch : 5 , phase : valid , loss : 0.6850489927572441
epoch : 6 , phase : train , loss : 0.6719609167035114
epoch : 6 , phase : valid , loss : 0.6545516308790982
epoch : 7 , phase : train , loss : 0.6423258949765303
epoch : 7 , phase : valid , loss : 0.626070957784818
epoch : 8 , phase : train , loss : 0.61464486179801
epoch : 8 , phase : valid , loss : 0.5994407061490827
epoch : 9 , phase : train , loss

epoch : 74 , phase : valid , loss : 0.1291319895738226
epoch : 75 , phase : train , loss : 0.12845528792651856
epoch : 75 , phase : valid , loss : 0.12746629014287994
epoch : 76 , phase : train , loss : 0.1268065752220303
epoch : 76 , phase : valid , loss : 0.1258409210130669
epoch : 77 , phase : train , loss : 0.12519721765602929
epoch : 77 , phase : valid , loss : 0.12425487814229916
epoch : 78 , phase : train , loss : 0.12362710187185923
epoch : 78 , phase : valid , loss : 0.1227066388611561
epoch : 79 , phase : train , loss : 0.12209401040953412
epoch : 79 , phase : valid , loss : 0.12119471499021241
epoch : 80 , phase : train , loss : 0.12059681224983078
epoch : 80 , phase : valid , loss : 0.11971755447968117
epoch : 81 , phase : train , loss : 0.11913438656288843
epoch : 81 , phase : valid , loss : 0.11827500915032349
epoch : 82 , phase : train , loss : 0.11770522532307615
epoch : 82 , phase : valid , loss : 0.11686482182356034
epoch : 83 , phase : train , loss : 0.11630877575748

epoch : 147 , phase : valid , loss : 0.06518460989679531
epoch : 148 , phase : train , loss : 0.0650149607096149
epoch : 148 , phase : valid , loss : 0.06473739026356529
epoch : 149 , phase : train , loss : 0.06457008741698522
epoch : 149 , phase : valid , loss : 0.06429603903915922
epoch : 150 , phase : train , loss : 0.06413119018538607
epoch : 150 , phase : valid , loss : 0.06386056197190963
epoch : 151 , phase : train , loss : 0.06369796906546266
epoch : 151 , phase : valid , loss : 0.06343087347553199
epoch : 152 , phase : train , loss : 0.06327043297290706
epoch : 152 , phase : valid , loss : 0.06300655456045419
epoch : 153 , phase : train , loss : 0.06284857490253264
epoch : 153 , phase : valid , loss : 0.062587931604224
epoch : 154 , phase : train , loss : 0.06243196753795497
epoch : 154 , phase : valid , loss : 0.06217476126076374
epoch : 155 , phase : train , loss : 0.06202088217094718
epoch : 155 , phase : valid , loss : 0.06176666414012548
epoch : 156 , phase : train , loss

In [19]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

print(f"""comprimento médio de código final no dataset de validação: {valid_loss[-1]}
(compare com a entropia do dataset de validação).""")

comprimento médio de código final no dataset de treino: 0.04802617473719189
(compare com a entropia do dataset de treino).
comprimento médio de código final no dataset de validação: 0.047868039456534145
(compare com a entropia do dataset de validação).


### Pesos aprendidos são aproximadamente os parâmetros usados para gerar os dados

In [20]:
for param in net.parameters():
    print(param.data)

tensor([[6.8337, 6.4164, 6.6655, 6.8767, 6.7930]], device='cuda:0')
tensor([23.5015], device='cuda:0')


In [21]:
C.T, C0

NameError: name 'C' is not defined